In [ ]:
import pandas as pd
import numpy as np

csv_data = pd.read_csv("./Data/wine.csv")
csv_data.head()

In [ ]:
wine_data = csv_data[["alcohol", "sugar", "pH"]]
wine_target = csv_data["class"]

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(wine_data, wine_target, test_size=0.2, random_state=42)

sub_data, val_data, sub_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(sub_data, sub_target)
print(dt_model.score(sub_data, sub_target))
print(dt_model.score(val_data, val_target))

### 교차검증
![교차검증](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FriP4b%2FbtqZlQyFueH%2FOJ1lk0AyTpEWlVMEbS0L01%2Fimg.png)

k개로 나눈 데이터셋의 부분집합들(한 칸씩)을 1번 fold, 2번 fold 라 부른다.

교차검증은 바로 그 fold 들을 차례대로 교차해 테스트 데이터로 사용한다는 말.

위 그림 예시에서는 첫 번째 모델 학습에서는 5번 fold를 테스트 데이터로 사용한다고 것이고, 다음 모델 학습에서는 4번 fold가 테스트 데이터이고, 그다음엔 3번 fold 이런 식으로 이어지면서 마지막 다섯 번째 모델 학습까지 차례대로 fold가 테스트 데이터로 교차되어 사용됨.

In [ ]:
from sklearn.model_selection import (교차검증 함수)
scores = cross_validate(dt_model, train_data, train_target)
print(scores)

In [ ]:
print(np.mean(scores["test_score"]))

## cross_validate : spliter
* 기본 5개의 폴더로 분리 
* 10개의 분리 : cv = 10
* cv=스플리트 객체
* cv=KFold() : 회귀모델
* cv=StratifiedKFold() : 분류모델

In [ ]:
from sklearn.model_selection import (교차검증 분류모델)
scores = cross_validate(dt_model, train_data, train_target, cv = StratifiedKFold())
print(np.mean(scores["test_score"]))

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt_model, train_data, train_target, cv = splitter)
print(np.mean(scores["test_score"]))

## min_impurity_decrease
* 부모노드와 자식노드의 불순도 차이 : 정보이득
* 불순도의 차이가 클수록 좋음
* 주어진 불순도의 최소값보다 작으면 가지분할(노드분할) 하지 않음

### n_jobs=n : n개의 코어를 사용하여 훈련
### n이 -1이면 이용가능한 모든코어 활용

In [ ]:
from sklearn.model_selection import (그리드서치 클래스))
params = {"min_impurity_decrease":[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs_model = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs_model.fit(train_data, train_target)

In [ ]:
dt = gs_model.best_estimator_
print(dt.score(train_data, train_target))

In [ ]:
print(gs_model.best_params_)

In [ ]:
print(gs_model.cv_results_["mean_test_score"])

In [ ]:
best_index = np.argmax(gs_model.cv_results_["mean_test_score"])
print(gs_model.cv_results_["params"][best_index])

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs_model = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs_model.fit(train_data, train_target)

In [ ]:
print(gs_model.best_params_)

In [ ]:
print(np.max(gs_model.cv_results_["mean_test_score"]))
print(gs_model.cv_results_["mean_test_score"].shape)

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import (랜덤서치 클래스)

rs_model = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
rs_model.fit(train_data, train_target)

In [ ]:
print(rs_model.best_params_)

In [ ]:
print(np.max(rs_model.cv_results_["mean_test_score"]))
print(rs_model.cv_results_["mean_test_score"].shape)

In [ ]:
dt =rs_model.best_estimator_
print(dt.score(test_data, test_target))